In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
##Loading the train data
train_data=pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#testing the hypothesis that all females survived
women=train_data.loc[train_data.Sex=="female"]
women.head()
rate=sum(women["Survived"])/len(women["Survived"])
print("female survival rate ",rate)
men=train_data.loc[train_data.Sex=="male"]
mrate=sum(men["Survived"])/len(men["Survived"])
print("men survival rate",mrate)


female survival rate  0.7420382165605095
men survival rate 0.18890814558058924


In [4]:
#now lets proceed with importing the modules required for training 
from sklearn import tree
from sklearn.preprocessing import LabelEncoder



In [5]:
## lets use a decision tree to predict the survivals 
##But first let`s clean the data

test_data  = pd.read_csv("/kaggle/input/titanic/test.csv")

clean = train_data.dropna()
clean = clean.drop_duplicates()
clean = clean.drop(["Name", "Ticket", "Cabin"], axis=1)

le_sex = LabelEncoder()
le_emb = LabelEncoder()

clean["Sex"] = le_sex.fit_transform(clean["Sex"])
clean["Embarked"] = le_emb.fit_transform(clean["Embarked"])


clean_test = test_data.drop(["Name", "Ticket", "Cabin"], axis=1)
clean_test["Age"] = clean_test["Age"].fillna(clean_test["Age"].median())
clean_test["Fare"] = clean_test["Fare"].fillna(clean_test["Fare"].median())
clean_test["Embarked"] = clean_test["Embarked"].fillna(clean_test["Embarked"].mode()[0])



clean_test["Sex"] = le_sex.transform(clean_test["Sex"])
clean_test["Embarked"] = le_emb.transform(clean_test["Embarked"])

In [6]:
## now lets divide data into info and target
from sklearn.model_selection import train_test_split
target=clean["Survived"]
target
info= clean.drop(["Survived"], axis=1)
info

#further divide the data into train and test 
info_train, info_test, target_train, target_test = train_test_split(
    info, target, test_size=0.2, random_state=42, stratify=target
)


In [7]:
##training and testing the model
model=tree.DecisionTreeClassifier()
model.fit(info_train,target_train)
model.score(info_test,target_test)


0.7567567567567568

In [8]:
#testing the model on real test data and creating the submssion file 

predictions = model.predict(clean_test)
output = pd.DataFrame({'PassengerId': clean_test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)


In [9]:
clean_test.shape[0]

418